In [1]:
import pickle
import networkx as nx
import random
import chime

In [2]:
network = pickle.load(open('new_weighted_network.pkl', 'rb')).copy()

### Functions

In [3]:
def randomAlgo(network):

    # Assuming G is your graph
    venues = nx.get_node_attributes(network, 'venues').values()

    # Flatten the list of venues
    flattened_venues = [venue for sublist in venues for venue in sublist]

    # Get unique venues
    unique_venues = set(flattened_venues)

    selected_nodes = {}
    for venue in unique_venues:
        # get nodes with this venue
        nodes = [n for n, v in nx.get_node_attributes(network, 'venues').items() if venue in v]
        
        # randomly select a node
        selected_node = random.choice(nodes)
        
        selected_nodes[venue] = selected_node

    return selected_nodes.values()

In [4]:
def remove_edges_based_on_project_network(expert_network, project_network):
    edges_to_remove = []

    for edge in expert_network.edges():
        node1_label = expert_network.nodes[edge[0]]['venues'][0]
        node2_label = expert_network.nodes[edge[1]]['venues'][0]

        if not project_network.has_edge(node1_label, node2_label):
            edges_to_remove.append(edge)

    expert_network.remove_edges_from(edges_to_remove)

    return expert_network

In [5]:
def compute_influence(graph):
    if graph is None:
        print("Error: Graph is None.")
        return None

    for node in graph.nodes:
        total_weight = sum(edge['weight']
                           for _, _, edge in graph.edges(node, data=True))
        num_papers = len(graph.nodes[node]["papers"])
        num_coauthors = len(graph.nodes[node]["coauthors"])
        influence = num_papers + 1.5 * num_coauthors + total_weight
        graph.nodes[node]['influence'] = influence

    # Scale scores to 100
    max_influence = max(graph.nodes[node]['influence'] for node in graph.nodes)
    scale_factor = 100 / max_influence

    for node in graph.nodes:
        graph.nodes[node]['influence'] *= scale_factor

    return graph

In [6]:
def get_top_node_per_venue(graph):
    top_nodes = {}
    nodes = []
    graph = compute_influence(graph)
    for node in graph.nodes:
        venue = graph.nodes[node]['venues'][0]
        if venue not in top_nodes:
            top_nodes[venue] = [node]
        elif graph.nodes[node]['influence'] > graph.nodes[top_nodes[venue][0]]['influence']:
            top_nodes[venue] = [node]
        elif graph.nodes[node]['influence'] == graph.nodes[top_nodes[venue][0]]['influence']:
            top_nodes[venue].append(node)
    for item in top_nodes.values():
        nodes.extend(item)
    return top_nodes, nodes, graph.subgraph([node for nodes in top_nodes.values() for node in nodes])


In [7]:
top_nodes = get_top_node_per_venue(network)
for venue, node in top_nodes[0].items():
    print(f"Venue: {venue}, Node: {node}")


Venue: IJCAI, Node: [13774]
Venue: AAAI, Node: [16253]
Venue: KDD, Node: [11204]
Venue: AAMAS, Node: [1184]
Venue: NIPS, Node: [8398]


In [8]:
def createProjectNetwork(list):
    project = nx.Graph()
    project.add_edges_from(list)
    return project

In [9]:
def sum_edge_weights(graph):
    total_weight = 0

    for _, _, data in graph.edges(data=True):
        if 'weight' in data:
            total_weight += data['weight']

    return total_weight

In [10]:
def monte_carlo(f, graph_G, graph_P, num_iter):
    comm_eff = 0
    # selected_nodes = set()
    for i in range(num_iter):
        best = f(graph_G, graph_P)
        # selected_nodes = selected_nodes.union(set(best.nodes))
        eff = sum_edge_weights(best)
        comm_eff = comm_eff + eff
    avg_comm_eff = comm_eff/num_iter
    # print(f"Selected nodes: {selected_nodes}")
    print(f"Average Communication efficiency is : {avg_comm_eff}")

    return round(avg_comm_eff, 2)

In [11]:
def randomGreedy(graph_G, graph_P):
    """
    This function implements a random greedy algorithm to find a subgraph of graph_G 
    that has the same number of nodes as graph_P. The algorithm starts with a random node 
    from graph_G and iteratively adds the node that maximizes the total edge weight of the subgraph.

    Parameters:
    graph_G (networkx.Graph): The graph to find the subgraph in.
    graph_P (networkx.Graph): The graph to match the number of nodes with.

    Returns:
    networkx.Graph: The resulting subgraph of graph_G.
    """
    if not graph_G or not graph_P or len(graph_P.nodes) > len(graph_G.nodes):
        print("Error: Invalid input graphs.")
        return None

    # Start with a random node from G
    subset = {random.choice(list(graph_G.nodes))}
    labels = [graph_G.nodes[next(iter(subset))]['venues'][0]]

    while len(subset) < len(graph_P.nodes):
        # Find the node that maximizes the total edge weight of the subgraph
        candidates = [(node, sum_edge_weights(graph_G.subgraph(list(subset) + [node])))
                      for node in set(graph_G.nodes) - subset
                      if graph_G.nodes[node]['venues'][0] not in labels]

        if not candidates:
            print("Warning: No suitable node found. Terminating the loop.")
            break

        # Add the best node to the subset
        best_node, _ = max(candidates, key=lambda x: x[1])
        subset.add(best_node)
        labels.append(graph_G.nodes[best_node]['venues'][0])

    return graph_G.subgraph(subset)

In [12]:
def influenceGreedy(graph_G, graph_P):
    """
    This function implements a greedy algorithm to find a subgraph of graph_G 
    that has the same number of nodes as graph_P. The algorithm starts with a random node 
    from the top nodes per venue in graph_G and iteratively adds the node that maximizes 
    the total edge weight of the subgraph.

    Parameters:
    graph_G (networkx.Graph): The graph to find the subgraph in.
    graph_P (networkx.Graph): The graph to match the number of nodes with.

    Returns:
    networkx.Graph: The resulting subgraph of graph_G.
    """
    if not graph_G or not graph_P or len(graph_P.nodes) > len(graph_G.nodes):
        print("Error: Invalid input graphs.")
        return None

    # Start with a random node from the top nodes per venue with rank 1
    top_nodes = get_top_node_per_venue(graph_G)
    key = None
    for node in top_nodes[1]:
        if graph_G.nodes[node]['rank'] == 1:
            key = node
            break

    if key is None:
        print("Warning: No suitable node found with rank 1. Using a random node instead.")
        key = random.choice(top_nodes[1])

    subset = {key}
    labels = [graph_G.nodes[key]['venues'][0]]

    while len(subset) < len(graph_P.nodes):
        # Find the node that maximizes the total edge weight of the subgraph
        candidates = [(node, sum_edge_weights(graph_G.subgraph(list(subset) + [node])))
                      for node in set(graph_G.nodes) - subset
                      if graph_G.nodes[node]['venues'][0] not in labels]

        if not candidates:
            print("Warning: No suitable node found. Terminating the loop.")
            break

        # Add the best node to the subset
        best_node, _ = max(candidates, key=lambda x: x[1])
        subset.add(best_node)
        labels.append(graph_G.nodes[best_node]['venues'][0])

    return graph_G.subgraph(subset)

In [13]:
def randomMonteCarlo(graph, num_iter):
    total_weight = 0

    for _ in range(num_iter):
        total_weight += sum_edge_weights(graph.subgraph(randomAlgo(graph)))

    avg_weight = round(total_weight / num_iter, 2)
    print(f"Using Random : {avg_weight}")
    return avg_weight

### Experiments

In [14]:
network = compute_influence(network)

In [15]:
pickle.dump(network, open('new_weighted_network.pkl', 'wb'))


In [16]:
top_10_authors = []
# Create subgraphs for each conference with the top 10 authors by the number of papers
conferences = []
for venue in nx.get_node_attributes(network, 'venues').values():
    conferences.extend(venue)

for conference in set(conferences):
    conference_authors = [author for author, data in network.nodes(data=True) if data['venues'][0] == conference]
    top_authors = sorted(conference_authors, key=lambda author: (network.nodes[author]['venues'][0], network.nodes[author]['influence']), reverse=True)[:10]
    for i, author in enumerate(top_authors):
        network.nodes[author]['rank'] = i + 1
    top_10_authors.extend(top_authors)

subgraph = network.subgraph(top_10_authors)

In [17]:
# get_top_node_per_venue(subgraph)

In [18]:
# Fully connected project network

project_1 = [('NIPS', 'IJCAI'), ('NIPS', 'AAAI'), ('NIPS', 'AAMAS'), ('NIPS', 'KDD'), ('IJCAI', 'AAAI'), ('IJCAI', 'AAMAS'), ('IJCAI', 'KDD'), ('AAAI', 'AAMAS'), ('AAAI', 'KDD'), ('AAMAS', 'KDD')]
project_1 = createProjectNetwork(project_1)
# Remove Edges to match project network
network_based_on_project_1 = remove_edges_based_on_project_network(subgraph.copy(), project_1)

In [19]:
# Star connected project network
project_2 = [('NIPS', 'KDD'), ('IJCAI', 'KDD'), ('AAAI', 'KDD'), ('AAMAS', 'KDD')]
project_2 = createProjectNetwork(project_2)
# Remove Edges to match project network
network_based_on_project_2 = remove_edges_based_on_project_network(subgraph.copy(), project_2)

In [20]:
# Chain connected project network
project_3 = [('NIPS', 'KDD'), ('IJCAI', 'KDD'), ('AAAI', 'IJCAI'), ('AAMAS', 'AAAI')]
project_3 = createProjectNetwork(project_3)
# Remove Edges to match project network
network_based_on_project_3 = remove_edges_based_on_project_network(subgraph.copy(), project_3)

In [21]:
random_coordinators = randomGreedy(network_based_on_project_1, project_1)
print("Random Greedy : project 1")
for node in random_coordinators.nodes:
    print(f'{node}: {random_coordinators.nodes[node]["venues"][0]} : {random_coordinators.nodes[node]["rank"]}')

Random Greedy : project 1
15968: IJCAI : 9
10213: AAAI : 3
9840: NIPS : 9
12532: AAMAS : 5
3575: KDD : 9


In [22]:
random_coordinators = randomGreedy(network_based_on_project_2, project_2)
print("Random Greedy : project 2")
for node in random_coordinators.nodes:
    print(f'{node}: {random_coordinators.nodes[node]["venues"][0]}: {random_coordinators.nodes[node]["rank"]}')

Random Greedy : project 2
4037: IJCAI: 7
5829: NIPS: 8
13548: KDD: 4
15665: AAMAS: 7
16253: AAAI: 1


In [23]:
random_coordinators = randomGreedy(network_based_on_project_3, project_3)
print("Random Greedy : project 3")
for node in random_coordinators.nodes:
    print(f'{node}: {random_coordinators.nodes[node]["venues"][0]}')

Random Greedy : project 3
11204: KDD
2828: AAAI
11469: NIPS
890: IJCAI
14783: AAMAS


In [24]:
influence_greedy_coordinators = influenceGreedy(network_based_on_project_1, project_1)
print("Influence Greedy : Project 1")
for node in influence_greedy_coordinators.nodes:
    print(f'{node}: {influence_greedy_coordinators.nodes[node]["venues"][0]}')

Influence Greedy : Project 1
2850: NIPS
11204: KDD
15665: AAMAS
729: AAAI
890: IJCAI


In [25]:
influence_greedy_coordinators = influenceGreedy(network_based_on_project_2, project_2)
print("Influence Greedy : Project 2")
for node in influence_greedy_coordinators.nodes:
    print(f'{node}: {influence_greedy_coordinators.nodes[node]["venues"][0]}: {influence_greedy_coordinators.nodes[node]["rank"]}')

Influence Greedy : Project 2
4037: IJCAI: 7
5829: NIPS: 8
13548: KDD: 4
15665: AAMAS: 7
16253: AAAI: 1


In [26]:
influence_greedy_coordinators = influenceGreedy(network_based_on_project_3, project_3)
print("Influence Greedy : Project 3")
for node in influence_greedy_coordinators.nodes:
    print(f'{node}: {influence_greedy_coordinators.nodes[node]["venues"][0]}')

Influence Greedy : Project 3
11204: KDD
11469: NIPS
15665: AAMAS
729: AAAI
890: IJCAI


In [27]:
top_nodes  = get_top_node_per_venue(network_based_on_project_2)[1]
for node in top_nodes:
    print(f'{node}: {network_based_on_project_2.nodes[node]["venues"][0]} : {network_based_on_project_2.nodes[node]["rank"]}')

6173: AAAI : 7
13876: AAMAS : 3
4037: IJCAI : 7
11469: NIPS : 10
13548: KDD : 4


In [28]:
nodes_with_aamas = [node for node, data in network_based_on_project_2.nodes(data=True) if 'AAMAS' in data['venues']]
nodes_with_aamas_sorted = sorted(nodes_with_aamas, key=lambda node: network_based_on_project_2.nodes[node]['rank'])

for node in nodes_with_aamas_sorted:
    venue = network_based_on_project_2.nodes[node]['venues'][0]
    rank = network_based_on_project_2.nodes[node]['rank']
    print(f"Node: {node}, Venue: {venue}, Rank: {rank}")


nodes_with_aamas = [node for node, data in network_based_on_project_2.nodes(data=True) if 'AAMAS' in data['venues']]
for node in nodes_with_aamas:
    print(f"Node: {node}, Data: {network_based_on_project_2.nodes[node]}")


Node: 1184, Venue: AAMAS, Rank: 1
Node: 14783, Venue: AAMAS, Rank: 2
Node: 13876, Venue: AAMAS, Rank: 3
Node: 14734, Venue: AAMAS, Rank: 4
Node: 12532, Venue: AAMAS, Rank: 5
Node: 10039, Venue: AAMAS, Rank: 6
Node: 6173, Venue: AAAI, Rank: 7
Node: 15665, Venue: AAMAS, Rank: 7
Node: 8926, Venue: AAMAS, Rank: 8
Node: 11586, Venue: AAMAS, Rank: 9
Node: 2789, Venue: AAMAS, Rank: 10
Node: 14734, Data: {'id': 14734, 'author': 'I. Starnes', 'coauthors': ['M. Feldhousen', 'V. Malhotra', 'X. Zhang', 'Larry Kerschberg', 'J. Cheng', 'C. Long', 'G. Emami', 'Alexander Brodsky', 'D. Cornwell'], 'venues': ['AAMAS'], 'papers': ['ACTIVE: agile coordinator testbed integrated virtual environment.'], 'num_of_papers': 1, 'influence': 18.252041491944382, 'rank': 4}
Node: 6173, Data: {'id': 6173, 'author': 'Christopher J. Carpenter', 'coauthors': ['Gaurav Naik', 'Pragnesh Jay Modi', 'Joseph Kopena', 'Duc N. Nguyen', 'Evan Sultanik', 'Robert N. Lass', 'Christopher Dugan', 'William C. Regli', 'Gaurav Naik', 'P

### Project 1 - Fully Connected

In [29]:
monte_carlo(randomGreedy,network_based_on_project_1, project_1, 10000)

Average Communication efficiency is : 780.0786


780.08

In [30]:
monte_carlo(influenceGreedy,network_based_on_project_1, project_1, 10000)

Average Communication efficiency is : 802.796


802.8

In [31]:
randomMonteCarlo(network_based_on_project_1, 10000)

Using Random : 480.32


480.32

In [32]:
sum_edge_weights(get_top_node_per_venue(network_based_on_project_1)[2])

624

### Project 2 - Star

In [33]:
monte_carlo(randomGreedy,network_based_on_project_2, project_2, 10000)

Average Communication efficiency is : 365.1517


365.15

In [34]:
monte_carlo(influenceGreedy,network_based_on_project_2, project_2, 10000)

Average Communication efficiency is : 381.0401


381.04

In [35]:
randomMonteCarlo(network_based_on_project_2, 10000)

Using Random : 209.46


209.46

In [36]:
sum_edge_weights(get_top_node_per_venue(network_based_on_project_2)[2])

312

### Project 3 - Chain

In [37]:
monte_carlo(randomGreedy,network_based_on_project_3, project_3, 10000)

Average Communication efficiency is : 362.2519


362.25

In [38]:
monte_carlo(influenceGreedy,network_based_on_project_3, project_3, 10000)

Average Communication efficiency is : 375.1056


375.11

In [39]:
randomMonteCarlo(network_based_on_project_3, 10000)

Using Random : 204.53


204.53

In [40]:
sum_edge_weights(get_top_node_per_venue(network_based_on_project_3)[2])

280

In [41]:
# import utils_2
# utils_2.process_paper_data('dblp_data.txt', 'net_100.pkl', 100)

In [42]:

# with open('dblp_data.txt', 'r', encoding='utf-8') as file:
#     # assuming each paper is separated by two newlines
#     papers_data = file.read(1000).split('\n\n')
#     print(papers_data[0])
